<a href="https://colab.research.google.com/github/danyl-dnl/chat-bot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json
import os
from IPython.display import display, Markdown

try:
    from google.colab import userdata
    API_KEY = userdata.get('GEMINI_API_KEY')
    if not API_KEY:
        print("Warning: GEMINI_API_KEY not found in Colab Secrets. Please add it.")
        print("Falling back to environment variable or empty string. API calls might fail.")
except ImportError:
    API_KEY = os.getenv("GEMINI_API_KEY", "")

API_URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"

chat_history = []

def send_message_to_gemini(prompt: str):
    global chat_history

    chat_history.append({"role": "user", "parts": [{"text": prompt}]})

    payload = {
        "contents": chat_history,
    }

    headers = {
        'Content-Type': 'application/json'
    }

    full_api_url = f"{API_URL}?key={API_KEY}"

    print("Chatbot: Thinking...")

    try:
        response = requests.post(full_api_url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()

        result = response.json()

        if result.get("candidates") and len(result["candidates"]) > 0 and \
           result["candidates"][0].get("content") and \
           result["candidates"][0]["content"].get("parts") and \
           len(result["candidates"][0]["content"]["parts"]) > 0:
            bot_response = result["candidates"][0]["content"]["parts"][0]["text"]
            chat_history.append({"role": "model", "parts": [{"text": bot_response}]})
            display(Markdown(f"**Chatbot:** {bot_response}"))
            return bot_response
        else:
            error_msg = "An error occurred: Could not get a response from the model."
            print(error_msg)
            print(f"Full API response: {json.dumps(result, indent=2)}")
            return error_msg

    except requests.exceptions.RequestException as e:
        error_msg = f"Error during API request: {e}"
        print(error_msg)
        return error_msg
    except json.JSONDecodeError as e:
        error_msg = f"Error decoding JSON response: {e}"
        print(error_msg)
        return error_msg

def run_interactive_chatbot_colab():
    global chat_history
    chat_history = []

    print("--- Starting Interactive Gemini Chatbot Session ---")
    print("Chatbot: Hello! Ask me anything. Type 'exit' to end.")

    while True:
        user_input = input("You: ").strip()

        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break

        if not user_input:
            continue

        send_message_to_gemini(user_input)
        print("-" * 30)

    print("--- End of Interactive Chatbot Session ---")

if __name__ == "__main__":
    print("This script is designed for Google Colab. Please run the `run_interactive_chatbot_colab()` function from a Colab cell.")
    print("Refer to the 'Example Usage in Colab' comments in the code.")


This script is designed for Google Colab. Please run the `run_interactive_chatbot_colab()` function from a Colab cell.
Refer to the 'Example Usage in Colab' comments in the code.


In [ ]:
run_interactive_chatbot_colab()

--- Starting Interactive Gemini Chatbot Session ---
Chatbot: Hello! Ask me anything. Type 'exit' to end.
